In [6]:
path='C:/Users/satya/Downloads/smsspamcollection/SMSSpamCollection'

In [29]:
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf


In [ ]:
df=pd.read_csv(path,sep='\t',names=['label','message'])

In [8]:
df.head(2)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [50]:
df_spam=df[df['label']=='spam']
df_ham=df[df['label']=='ham']


In [51]:
df_ham_downsampled=df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 3)

In [52]:
df=pd.concat([df_spam,df_ham_downsampled])

In [53]:
df.head(2)

,label,message,spam
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1


In [9]:
df['spam']=df['label'].apply(lambda x: 1 if x=='spam' else 0)

In [10]:
df.head(2)

,label,message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0


In [54]:
from sklearn.model_selection import train_test_split

In [55]:
x_train,x_test,y_train,y_test=train_test_split(df['message'],df['spam'],test_size=0.4)

In [56]:
x_train.head(1)

385    Double mins and txts 4 6months FREE Bluetooth ...
Name: message, dtype: object

In [57]:
preprocess_url='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
bert_preprocess_model=hub.KerasLayer(preprocess_url)
bert_encoder=hub.KerasLayer(encoder_url)

In [58]:
def get_sentence_embedding(sentences):
    preprocess_text=bert_preprocess_model(sentences)
    return bert_encoder(preprocess_text)['pooled_output']

In [59]:
e=get_sentence_embedding(['banana',
                         'grapes',
                         'mango',
                          'elon-musk',
                          'jeff'
                         ])

In [60]:
#bert layers
test_input=tf.keras.layers.Input(shape=(),dtype=tf.string)
preprocessed_text=bert_preprocess_model(test_input)
outputs=bert_encoder(preprocessed_text)
#NN layers
l=tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
l=tf.keras.layers.Dense(1,activation='sigmoid')(l)

#model final
model=tf.keras.Model(inputs=[test_input],outputs=l)


In [61]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_mask': (Non  0           ['input_8[0][0]']                
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [62]:
metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]

In [63]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=metrics)
model.fit(x_train,y_train, validation_split=0.25, epochs=10)

Epoch 1/10
21/21 [==============================] - 86s 4s/step - loss: 0.7319 - accuracy: 0.4821 - precision: 0.4790 - recall: 0.4418 - val_loss: 0.6516 - val_accuracy: 0.5312 - val_precision: 1.0000 - val_recall: 0.0367
Epoch 2/10
21/21 [==============================] - 73s 4s/step - loss: 0.6247 - accuracy: 0.6771 - precision: 0.6867 - recall: 0.6478 - val_loss: 0.5615 - val_accuracy: 0.8304 - val_precision: 0.8318 - val_recall: 0.8165
Epoch 3/10
21/21 [==============================] - 75s 4s/step - loss: 0.5570 - accuracy: 0.7753 - precision: 0.7447 - recall: 0.8358 - val_loss: 0.5122 - val_accuracy: 0.8348 - val_precision: 0.9615 - val_recall: 0.6881
Epoch 4/10
21/21 [==============================] - 74s 4s/step - loss: 0.4992 - accuracy: 0.8333 - precision: 0.8348 - recall: 0.8299 - val_loss: 0.4576 - val_accuracy: 0.8795 - val_precision: 0.9100 - val_recall: 0.8349
Epoch 5/10
21/21 [==============================] - 79s 4s/step - loss: 0.4617 - accuracy: 0.8423 - precision: 0

In [67]:
y_pred=model.predict(x_test)


19/19 [==============================] - 49s 3s/step


In [68]:
import numpy as np 
y_pred=np.argmax(y_pred,axis=1)

In [69]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred,y_test)

array([[295, 303],
       [  0,   0]], dtype=int64)